These are a list of problems where parsing is slow in CVXPY.

The first four are all from the Convex.jl paper.

Be careful when profiling CVXPY that you create a new problem each time.
When you solve a problem it caches the cone program matrices, so if you solve it again it doesn't run the matrix stuffing algorithm.

In [48]:
# Preparation:
from cvxpy import *
import numpy

In [54]:
# Summation.
n = 10000
x = Variable()
e = 0
for i in range(n):
    e = e + x
p = Problem(Minimize(norm(e-1,2)), [x>=0])

In [55]:
# baseline cvxpy
settings.USE_CVXCANON = False

In [56]:
%%timeit
# This generates the cone program matrices but doesn't solve the cone program.
# 99% of the time here is spent in the matrix stuffing algorithm (going from LinOps to the final matrix).
p = Problem(Minimize(norm(e-1,2)), [x>=0])
p.get_problem_data(ECOS)

1 loops, best of 3: 6.71 s per loop


In [59]:
# CVXPY backed with CVXCanon
settings.USE_CVXCANON = True

In [60]:
%%timeit
p = Problem(Minimize(norm(e-1,2)), [x>=0])
p.get_problem_data(ECOS)

Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
1 loops, best of 3: 543 ms per loop


You can replace %%timeit with %%prun to profile the script.

In [7]:
# Indexing.
n = 10000
x = Variable(n)
e = 0
for i in range(n):
    e += x[i];

In [8]:
# baseline cvxpy
settings.USE_CVXCANON = False

In [9]:
%%timeit
p = Problem(Minimize(norm(e-1,2)), [x>=0])
p.get_problem_data(ECOS)

1 loops, best of 3: 18 s per loop


In [10]:
# CVXPY backed with CVXCanon
settings.USE_CVXCANON = True

In [11]:
%%timeit
p = Problem(Minimize(norm(e-1,2)), [x>=0])
p.get_problem_data(ECOS)

Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
1 loops, best of 3: 7.94 s per loop


In [12]:
# Transpose
n = 500
A = numpy.random.randn(n,n)

In [13]:
# baseline cvxpy
settings.USE_CVXCANON = False

In [14]:
%%timeit
X = Variable(n,n)
p = Problem(Minimize(norm(X.T-A,'fro')), [X[1,1] == 1])
p.get_problem_data(ECOS)

1 loops, best of 3: 468 ms per loop


In [15]:
# CVXPY backed with CVXCanon
settings.USE_CVXCANON = True

In [16]:
%%timeit
X = Variable(n,n)
p = Problem(Minimize(norm(X.T-A,'fro')), [X[1,1] == 1])
p.get_problem_data(ECOS)

Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
1 loops, best of 3: 215 ms per loop


In [17]:
# Matrix constraint.
# CVXPY actually does a pretty good job with this one.
# Convex.jl and CVX are slower (at least when they were profiled for the paper).
n = 500
A = numpy.random.randn(n,n)
B = numpy.random.randn(n,n)

In [18]:
# baseline cvxpy
settings.USE_CVXCANON = False

In [19]:
%%timeit
X = Variable(n,n)
p = Problem(Minimize(norm(X-A,'fro')), [X == B])
p.get_problem_data(ECOS)

1 loops, best of 3: 349 ms per loop


In [20]:
# CVXPY backed with CVXCanon
settings.USE_CVXCANON = True

In [21]:
%%timeit
X = Variable(n,n)
p = Problem(Minimize(norm(X-A,'fro')), [X == B])
p.get_problem_data(ECOS)

Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
10 loops, best of 3: 168 ms per loop


In [22]:
# Matrix product.
# This one is a bit different, because the issue is that the coefficient for A.T*X*A has n^4 nonzeros.
# A fix is to introduce the variable A.T*X = Y, and rewrite A.T*X*A as Y*A. 
# This will only add 2n^3 nonzeros.
n = 50
A = numpy.random.randn(n,n)

In [23]:
# baseline cvxpy
settings.USE_CVXCANON = False

In [24]:
%%timeit
X = Variable(n,n)
p = Problem(Minimize(norm(X,'fro')), [A.T*X*A >= 1])
p.get_problem_data(ECOS)

1 loops, best of 3: 4.57 s per loop


In [25]:
# CVXPY backed with CVXCanon
settings.USE_CVXCANON = True

In [26]:
%%timeit
X = Variable(n,n)
p = Problem(Minimize(norm(X,'fro')), [A.T*X*A >= 1])
p.get_problem_data(ECOS)

Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
1 loops, best of 3: 1.96 s per loop


In [43]:
# SVM with indexing.
def gen_data(n):
    pos = numpy.random.multivariate_normal([1.0,2.0],numpy.eye(2),size=n)
    neg = numpy.random.multivariate_normal([-1.0,1.0],numpy.eye(2),size=n)
    return pos, neg

N = 2
C = 10
pos, neg = gen_data(500)

w = Variable(N)
b = Variable()
xi_pos = Variable(pos.shape[0])
xi_neg = Variable(neg.shape[0])
cost = sum_squares(w) + C*sum_entries(xi_pos) + C*sum_entries(xi_neg)
constrs = []
for j in range(pos.shape[0]):
    constrs += [w.T*pos[j,:] - b >= 1 - xi_pos[j]]
    
for j in range(neg.shape[0]):
    constrs += [-(w.T*neg[j,:] - b) >= 1 - xi_neg[j]]

In [44]:
# baseline cvxpy
settings.USE_CVXCANON = False

In [45]:
%%timeit
p = Problem(Minimize(cost), constrs)
p.get_problem_data(ECOS)

1 loops, best of 3: 5.84 s per loop


In [46]:
# CVXPY backed with CVXCanon
settings.USE_CVXCANON = True

In [47]:
%%timeit
p = Problem(Minimize(cost), constrs)
p.get_problem_data(ECOS)

Using CVXCANON
Using CVXCANON
Using CVXCANON
Using CVXCANON
1 loops, best of 3: 637 ms per loop
